# Type et méthodes

JULIA possède un système de type et de méthode qui lui confère une approche objet.
La fonction typeof() renvoie le type d'une variable de base Int32, Float64... JULIA est conçu pour permettre facilement d'étendre l'environnement à de nouveaux type de variable.

Le types sont organisés suivant un hiérarchie comme on peut le voir sur l'arborescence partielle ci-dessous

(arborescence générée à l'aide de https://github.com/tanmaykm/julia_types/blob/master/julia_types.jl)

Remarquez "abstract" et "concrete" dans cette arborescence.

# Méthodes

A chaque fonction est associée une méthode dépendante du type d'entrée comme dans ce qui suit suivant que l'entrée soit un entier ou pas.

In [1]:
function f(x::Any)
    gamma(x+1)
end

f (generic function with 1 method)

In [2]:
function f(n::Integer)
    factorial(n)
end

f (generic function with 2 methods)

In [3]:
f(3.0)

6.0

In [4]:
f(3)

6

In [5]:
f(im)

0.4980156681183314 - 0.15494982830177254im

In [6]:
f(-2)

LoadError: LoadError: DomainError:
while loading In[6], in expression starting on line 1

In [7]:
factorial(sqrt(2))

1.2538154806428916

In [8]:
f(sqrt(2))

1.2538154806428916

# Construction d'un nouveau Type de variable

En premier lieu il faut définir un type abstrait puis une instance sous-hiérarchiquement concrète :

In [9]:
abstract Grid # juste en dessous de Any
type Grid1d <: Grid
    debut::Float64
    fin::Float64
    n::Int32
end

In [10]:
a=Grid1d(0,1,2)

Grid1d(0.0,1.0,2)

In [11]:
a.debut

0.0

In [12]:
a.fin

1.0

In [13]:
a.n

2

# Surcharge des opérateurs

La surcharge des opérations usuelles se fait en définissant une nouvelle méthode associé au nouveau type pour chaque opérateur, commençons par surcharger l'affichage à l'écran de notre nouveau type. Pour cela on va ajouter une méthode à la fonction "show"

In [14]:
function Base.show(io::IO,g::Grid1d)
    print(io, "Grid 1d : début $(g.debut) , fin $(g.fin) , $(g.n) éléments\n")
end

show (generic function with 105 methods)

In [15]:
Base.show(a)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [34]:
println(a)

Grid 1d : début 0.0 , fin 1.0 , 6 éléments



## Addition, soustraction ...

Ces fonctions sont de la forme +(), -() c'est à dire

In [17]:
function +(g::Grid1d,n::Integer)
    g.n+=n
    return g
end

+ (generic function with 172 methods)

In [18]:
a=Grid1d(0,1,2)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [19]:
a+2

Grid 1d : début 0.0 , fin 1.0 , 4 éléments


In [20]:
a+=1

Grid 1d : début 0.0 , fin 1.0 , 5 éléments


Attention l'addition n'est pas forcément commutative !

In [21]:
2+a

LoadError: LoadError: MethodError: `+` has no method matching +(::Int64, ::Grid1d)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +(::Int64, !Matched::Int64)
  +(::Integer, !Matched::Ptr{T})
  ...
while loading In[21], in expression starting on line 1

ni unaire !

In [35]:
+a

LoadError: LoadError: MethodError: `+` has no method matching +(::Grid1d)
Closest candidates are:
  +(::Grid1d, !Matched::Integer)
  +(::Any, !Matched::Any, !Matched::Any, !Matched::Any...)
while loading In[35], in expression starting on line 1

Notez le message d'erreur qui est très claire !

In [24]:
a+[1,2]

LoadError: LoadError: MethodError: `+` has no method matching +(::Grid1d, ::Array{Int64,1})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +{S,T}(!Matched::Range{S}, ::AbstractArray{T,N})
  +(!Matched::SparseMatrixCSC{Tv,Ti<:Integer}, ::Array{T,N})
  ...
while loading In[24], in expression starting on line 1

## Autres surcharges

Toutes les fonctions usuelles sont surchargeable sans limite : size(); det() ...

In [36]:
function Base.size(g::Grid)
    return g.n
end

size (generic function with 63 methods)

In [37]:
size(a)

6

In [39]:
function Base.det(g::Grid1d)
    g.fin-g.debut
end 

det (generic function with 19 methods)

In [41]:
det(a)

1.0

# Type et constructeurs

Chaque langage "objet" définit un constructeur pour ces objets. Nous avons déjà utiliser un constructeur générique qui rempli chaque champ du nouveau type. Il est possible de faire une variante suivant le nombre d'arguments d'entrée et de leur type 

In [42]:
abstract Grid # juste en dessous de Any
type Grid1d <: Grid
    debut::Float64
    fin::Float64
    n::Int32
    
    # constructeurs par défaut sans argument
    function Grid1d()
        new(0,0,0)
    end
    
    # constructeurs par défaut avec argument
    function Grid1d(a,b,c)
        if c<=0
            error("pas possible")
        else
            new(a,b,c)
        end
    end
end

In [30]:
b=Grid1d(0,1,-1)

LoadError: LoadError: pas possible
while loading In[30], in expression starting on line 1

Il devient possible de déterminer un constructeurs pour différentes entrées.

Il faut au préalable bien penser sa hiérarchie de type et écrire autant de fonctions constructeurs que de cas d'initialisation du nouveau type.

# Les Itérateurs

Il est possible sur un type nouveau de définir un itérateur, comme ici de parcourrir les points de la grille, définissons (surchargeons) de nouvelles fonctions ou plutôt méthodes : 

In [102]:
Base.start(a::Grid1d) = 1

function Base.next(a::Grid1d, state)
    if state == 1
        return (a.debut,2)
    else
        return (a.debut+(a.fin-a.debut)*(state-1)/a.n , state+1)
    end
end

Base.done(a::Grid1d, state) = state > a.n +1



done (generic function with 52 methods)

In [103]:
a=Grid1d(0,1,10)

Grid 1d : début 0.0 , fin 1.0 , 10 éléments


In [104]:
start(a)

1

In [105]:
next(a,0)

(-0.1,1)

In [106]:
for i in a
    println(i)
end

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


Il devient possible de construire des itérateurs sur une grille 2d, 3d renvoyant les coordonnées des points de la grille... Mais on peut imaginer sur une triangulation etc... 